In [1]:
%matplotlib widget

import os
import numpy as np
from scipy.stats import median_abs_deviation as mad
from astropy.stats.biweight import biweight_location
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

from lvmdrp.utils.examples import fetch_example_data, get_frames_metadata, fix_lamps_metadata, get_masters_metadata

from lvmdrp.core.plot import plot_strips
from lvmdrp.core import rss, image, spectrum1d
from lvmdrp.functions import imageMethod, rssMethod

plt.style.use("seaborn-v0_8-talk")

In [2]:
# TODO: take a look at the bias subtracted bias

In [3]:
# define input data directory
data_path = os.path.abspath(os.path.join("..", "data"))

# let's create the output directory
output_path = "./data"
os.makedirs(output_path, exist_ok=True)

# create processed frames path template
out_main_path = os.path.join(output_path, "lvm-{kind}-{camera}-{expnum}.fits")
out_calib_path = os.path.join(output_path, "lvm-{mjd}-{kind}-{camera}-{exptime}.fits")

# overwrite or not
OVERWRITE = False

In [4]:
# extract metadata
frames_table = get_frames_metadata(path=os.path.join(data_path, "lco_com"), ignore_cache=False).to_pandas()
frames_table = frames_table.loc[~frames_table.mjd.isin([60008,60009,60037,60038])]
# fix arc lamps metadata
fix_lamps_metadata(metadata=frames_table, lamp_names="argon neon ldls hgne xenon krypton".split())

[INFO]: loading cached metadata from '/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examples/data/lco_com/frames_table.pkl'


In [5]:
# filter frames with continuum emission in the fibers
target_frames = frames_table.query("(imagetyp=='arc' & ldls & not (argon|neon|xenon|hgne|krypton)) | imagetyp=='object'")
target_frames = target_frames.sort_values("camera").reset_index(drop=True)
target_frames

,imagetyp,spec,mjd,camera,expnum,exptime,argon,neon,ldls,hgne,xenon,krypton,path
0,object,sp1,60041,b1,00000679,30.0,False,False,False,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
1,object,sp1,60040,b1,00000607,900.0,False,False,False,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
2,object,sp1,60039,b1,00000509,120.0,False,False,False,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
3,object,sp1,60040,b1,00000641,60.0,False,False,False,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
4,object,sp1,60010,b1,00000174,180.0,False,False,False,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,object,sp3,60039,z3,00000567,15.0,False,False,True,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
677,object,sp3,60039,z3,00000509,120.0,False,False,False,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
678,object,sp3,60039,z3,00000590,180.0,False,False,True,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...
679,object,sp3,60039,z3,00000504,120.0,False,False,False,False,False,False,/home/mejia/Research/UNAM/lvm-drp/lvmdrp/examp...


In [6]:
mjds = [60041]

table = pd.DataFrame(columns=[
    "frame",
    "mjd",
    "camera",
    "exposure",
    "obstime",
    "exptime",
    "cnt_med1",
    "cnt_med2",
    "cnt_med3",
    "cnt_med4",
    "cnt_std1",
    "cnt_std2",
    "cnt_std3",
    "cnt_std4",
    "cnt_mad1",
    "cnt_mad2",
    "cnt_mad3",
    "cnt_mad4",
    "cnt_mst1",
    "cnt_mst2",
    "cnt_mst3",
    "cnt_mst4",
    "rdnoise1",
    "rdnoise2",
    "rdnoise3",
    "rdnoise4",
    "gain1",
    "gain2",
    "gain3",
    "gain4",
])
median_os = []
biweight_os = []
for mjd in tqdm(mjds, desc="MJD", unit="mjd"):
    frame_groups = target_frames.loc[target_frames.mjd==mjd].groupby("camera")

    for camera in tqdm(frame_groups.groups, desc="camera", unit="camera"):
        frames_camera = frame_groups.get_group(camera).sort_values("mjd")
        
        for i, frame in tqdm(frames_camera.iterrows(), total=len(frames_camera), desc="frame", unit="frame"):
            img = image.loadImage(frame.path)

            # extract header metadata
            frame_name = img._header["FILENAME"].replace(".fits.gz", "")
            table.loc[i, "frame"] = frame_name
            table.loc[i, "mjd"] = mjd
            table.loc[i, "camera"] = camera
            table.loc[i, "exposure"] = img._header["EXPOSURE"]
            table.loc[i, "obstime"] = img._header["OBSTIME"]
            table.loc[i, "exptime"] = img._header["EXPTIME"]

            # compute statistics & plot quadrant strips
            median_quads = []
            bi_quads = []
            for j in range(4):
                quad = img.getSection(imageMethod.DEFAULT_BIASSEC[j])
                # quad *= img._header[f"GAIN{j+1}"]
                table.loc[i, f"cnt_med{j+1}"] = np.median(quad._data)
                table.loc[i, f"cnt_std{j+1}"] = np.std(quad._data)
                table.loc[i, f"cnt_mad{j+1}"] = mad(quad._data, axis=None)
                table.loc[i, f"cnt_mst{j+1}"] = np.mean(np.std(quad._data, axis=1))
                table.loc[i, f"rdnoise{j+1}"] = img._header[f"RDNOISE{j+1}"]
                table.loc[i, f"rdnoise{j+1}"] = img._header[f"GAIN{j+1}"]
                median_quads.append(np.median(quad._data, axis=1))
                bi_quads.append(biweight_location(quad._data, axis=1))
                

                strip_med = np.median(quad._data, axis=1)
                strip_mad = mad(quad._data, axis=1)
                strip_pix = np.arange(strip_med.size)
            median_os.append(median_quads)
            biweight_os.append(bi_quads)

# further processing of timeseries
table.obstime = pd.to_datetime(table.obstime)
table.set_index("obstime", inplace=True)
table.sort_index(inplace=True)

biweight_os = np.array(biweight_os)
median_os = np.array(median_os)
os_scale = np.median(median_os, axis=2)

MJD:   0%|          | 0/1 [00:00<?, ?mjd/s]

camera:   0%|          | 0/6 [00:00<?, ?camera/s]

frame:   0%|          | 0/28 [00:00<?, ?frame/s]

frame:   0%|          | 0/28 [00:00<?, ?frame/s]

frame:   0%|          | 0/28 [00:00<?, ?frame/s]

frame:   0%|          | 0/28 [00:00<?, ?frame/s]

frame:   0%|          | 0/28 [00:00<?, ?frame/s]

frame:   0%|          | 0/28 [00:00<?, ?frame/s]

In [7]:
plt.switch_backend("TkAgg")
camera_groups = table.reset_index().groupby(["camera"])
groups = camera_groups.groups


for cam in groups:
    cam_idx = groups[cam]

    obstime = camera_groups.get_group(cam).obstime.dt.to_pydatetime()
    reftime = obstime.min()

    # fig_h, axs_h = plt.subplots(2, 2, figsize=(10,15))
    # axs_h = axs_h.flatten()
    fig, axs = plt.subplots(2, 2, figsize=(15,5), sharex=True, sharey=True)
    axs = axs.flatten()
    for iquad in range(4):
        x = np.tile(np.arange(biweight_os.shape[2]), cam_idx.size).reshape((cam_idx.size, biweight_os.shape[2]))
        y = biweight_os[cam_idx, iquad, :] - os_scale[cam_idx, iquad, None]
        z = reftime - np.tile(obstime, biweight_os.shape[2]).reshape(biweight_os.shape[2], cam_idx.size).T
        z = np.abs(z.astype(np.timedelta64(1, "h")))

        sigma = camera_groups.get_group(cam)[f"rdnoise{iquad+1}"].iloc[0] / 2.6 / np.sqrt(17)
        xgauss = np.linspace(-5*sigma, 5*sigma, 50)
        # axs_h[iquad].hist(y.flatten(), bins=10, range=(-5*sigma, 5*sigma), density=True)
        # axs_h[iquad].plot(xgauss, np.exp(-0.5*(xgauss/sigma)**2))
        # axs_h[iquad].set_xlim(-5*sigma, 5*sigma)
        # axs_h[iquad].set_title(f"quadrant {iquad+1}")
        
        sc = axs[iquad].scatter(x, y, c=z, lw=0, s=5, cmap=plt.cm.rainbow)
        cb = plt.colorbar(sc)
        cb.set_label(f"hours since")
        
        axs[iquad].axhline(np.std(y, axis=None), ls="-", color="tab:blue")
        axs[iquad].axhline(camera_groups.get_group(cam)[f"rdnoise{iquad+1}"].iloc[0]/np.sqrt(17), ls="-", color="tab:red")
        axs[iquad].set_title(f"quadrant {iquad+1}", loc="left")
        axs[iquad].set_axisbelow(True)
        axs[iquad].grid(color="0.9", axis="y")
        
    axs[iquad].set_ylim(-10, 10)
    # fig_h.suptitle(f"camera = {cam}")
    fig.suptitle(f"camera = {cam}")
    fig.supxlabel("Y (pixel)")
    fig.supylabel("counts (ADU)")
    fig.tight_layout()
    fig.savefig(f"biweight_{cam}.png", bbox_inches="tight")
plt.show()